In [10]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import pickle
from src.modules.models import xgboost_reg, ensemble_feature_importance, regmodel_evaluation

## Load model

In [11]:
# load the model from disk
filename = 'output/xgb_model_012_noscale.pickle'

xgb_model = pickle.load(open(filename, 'rb'))
print(xgb_model)

{'XGBoost_Reg': XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.3, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)}


In [12]:
pd.set_option('display.max_columns', 100)
X_df = pd.read_csv('data/Test/test.csv')
print(X_df.shape)
X_df.head()

(150623, 83)


,mkt_unique_carrier,mkt_carrier_fl_num,tail_num,op_carrier_fl_num,origin_airport_id,origin,dest_airport_id,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day,day_of_week,week_of_year,crs_dep_hour,crs_arr_hour,month_avg_arr_delay,month_avg_air_time,month_avg_carrier_delay,month_avg_weather_delay,month_avg_nas_delay,month_avg_security_delay,month_avg_late_aircraft_delay,day_of_week_avg_arr_delay,day_of_week_avg_air_time,day_of_week_avg_carrier_delay,day_of_week_avg_weather_delay,day_of_week_avg_nas_delay,day_of_week_avg_security_delay,day_of_week_avg_late_aircraft_delay,arr_hour_avg_arr_delay,arr_hour_avg_air_time,arr_hour_avg_carrier_delay,arr_hour_avg_weather_delay,arr_hour_avg_nas_delay,arr_hour_avg_security_delay,arr_hour_avg_late_aircraft_delay,origin_total_flights,dest_total_flights,month_flight_avg_seats,month_flight_avg_passengers,carrier_month_seats,carrier_month_passengers,carrier_month_avg_seats,carrier_month_avg_passengers,month_avg_fuel_comsumption,orig_airport_month_flight_seats,orig_airport_month_passengers,dest_airport_month_flight_seats,dest_airport_month_passengers,origin_min_temp,origin_max_temp,origin_avg_temp,origin_total_snow_cm,origin_windspeed_kmph,origin_precipitation_mm,origin_humidity,origin_visibility,origin_cloud_cover,origin_heat_index_c,origin_wind_chill_c,origin_wind_gust_kmph,origin_feels_like,origin_weather_description,origin_pressure,dest_min_temp,dest_max_temp,dest_avg_temp,dest_total_snow_cm,dest_windspeed_kmph,dest_precipitation_mm,dest_humidity,dest_visibility,dest_cloud_cover,dest_heat_index_c,dest_wind_chill_c,dest_wind_gust_kmph,dest_feels_like,dest_weather_description,dest_pressure
0,WN,5888,N951WN,5888,13891,ONT,14771,SFO,1810,1945,95,363,2020,1,1,3,1,18,19,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,57.125000,1.478261,0.00,42.521739,0.0,9.956522,5.258824,56.623529,18.421053,0.0,25.368421,0.0,26.473684,7086.0,57334.0,152.074203,113.048765,79269676,58102809,154.052359,113.739038,156532592.0,2303449.0,1741130.0,24365383.0,18967854.0,10.0,18.0,15.0,0.0,4.0,0.0,34.0,10.0,5.0,14.0,14.0,7.0,14.0,Partly cloudy,1017.0,11.0,14.0,13.0,0.0,12.0,0.0,83.0,10.0,29.0,13.0,12.0,17.0,12.0,Partly cloudy,1021.0
1,WN,6276,N467WN,6276,13891,ONT,14771,SFO,1150,1320,90,363,2020,1,1,3,1,11,13,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,57.125000,1.478261,0.00,42.521739,0.0,9.956522,21.142857,59.142857,0.000000,0.0,17.500000,0.0,77.000000,7086.0,57334.0,152.074203,113.048765,79269676,58102809,154.052359,113.739038,156532592.0,2303449.0,1741130.0,24365383.0,18967854.0,10.0,18.0,15.0,0.0,4.0,0.0,34.0,10.0,5.0,14.0,14.0,7.0,14.0,Partly cloudy,1017.0,11.0,14.0,13.0,0.0,12.0,0.0,83.0,10.0,29.0,13.0,12.0,17.0,12.0,Partly cloudy,1021.0
2,WN,4598,N7885A,4598,13891,ONT,14831,SJC,2020,2130,70,333,2020,1,1,3,1,20,21,-0.467742,53.842742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,52.297009,9.200000,0.72,3.920000,0.0,33.140000,8.811881,51.735974,7.871429,0.0,2.728571,0.0,47.800000,7086.0,20962.0,155.201183,98.481989,79269676,58102809,154.052359,113.739038,156532592.0,2303449.0,1741130.0,6173045.0,4355999.0,10.0,18.0,15.0,0.0,4.0,0.0,34.0,10.0,5.0,14.0,14.0,7.0,14.0,Partly cloudy,1017.0,8.0,16.0,12.0,0.0,6.0,0.0,71.0,10.0,44.0,11.0,11.0,8.0,11.0,Overcast,1021.0
3,WN,4761,N551WN,4761,13891,ONT,14831,SJC,1340,1455,75,333,2020,1,1,3,1,13,14,-0.467742,53.842742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,52.297009,9.200000,0.72,3.920000,0.0,33.140000,2.074074,51.700000,15.717391,0.0,2.239130,0.0,21.956522,7086.0,20962.0,155.201183,98.481989,79269676,58102809,154.052359,113.739038,156532592.0,2303449.0,1741130.0,6173045.0,4355999.0,10.0,18.0,15.0,0.0,4.0,0.0,34.0,10.0,5.0,14.0,14.0,7.0,14.0,Partly cloudy,1017.0,8.0,16.0,12.0,0.0,6.0,0.0,71.0,10.0,44.0,11.0,11.0,8.0,11.0,Overcast,1021.0
4,WN,5162,N968WN,5162,13891,ONT,14831,SJC,915,1035,80,333,2020,1,1,3,1,9,10,-0.467742,53.842742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,52.297009,9.200000,

In [13]:

X_df = X_df.loc[:, ['mkt_unique_carrier', 'crs_elapsed_time', 'year', 'month', 'day', 'day_of_week', 'crs_dep_hour', 'crs_arr_hour', 'month_avg_arr_delay',
             'month_avg_carrier_delay', 'month_avg_weather_delay', 'month_avg_nas_delay', 'month_avg_security_delay', 'month_avg_late_aircraft_delay', 
             'day_of_week_avg_arr_delay', 'day_of_week_avg_carrier_delay', 'day_of_week_avg_weather_delay', 'day_of_week_avg_nas_delay', 
             'day_of_week_avg_security_delay', 'day_of_week_avg_late_aircraft_delay', 'arr_hour_avg_arr_delay', 'arr_hour_avg_carrier_delay', 
             'arr_hour_avg_weather_delay', 'arr_hour_avg_nas_delay', 'arr_hour_avg_security_delay', 'arr_hour_avg_late_aircraft_delay', 'origin_total_flights', 
             'dest_total_flights', 'month_flight_avg_seats', 'carrier_month_seats', 'carrier_month_avg_seats', 'origin_min_temp', 'origin_total_snow_cm', 
             'origin_windspeed_kmph', 'origin_precipitation_mm', 'origin_humidity', 'origin_visibility', 'origin_cloud_cover', 'origin_pressure', 'dest_min_temp', 
             'dest_total_snow_cm', 'dest_windspeed_kmph', 'dest_precipitation_mm', 'dest_humidity', 'dest_visibility', 'dest_cloud_cover', 'dest_pressure']]
print(X_df.shape)
X_df.head()

(150623, 47)


,mkt_unique_carrier,crs_elapsed_time,year,month,day,day_of_week,crs_dep_hour,crs_arr_hour,month_avg_arr_delay,month_avg_carrier_delay,month_avg_weather_delay,month_avg_nas_delay,month_avg_security_delay,month_avg_late_aircraft_delay,day_of_week_avg_arr_delay,day_of_week_avg_carrier_delay,day_of_week_avg_weather_delay,day_of_week_avg_nas_delay,day_of_week_avg_security_delay,day_of_week_avg_late_aircraft_delay,arr_hour_avg_arr_delay,arr_hour_avg_carrier_delay,arr_hour_avg_weather_delay,arr_hour_avg_nas_delay,arr_hour_avg_security_delay,arr_hour_avg_late_aircraft_delay,origin_total_flights,dest_total_flights,month_flight_avg_seats,carrier_month_seats,carrier_month_avg_seats,origin_min_temp,origin_total_snow_cm,origin_windspeed_kmph,origin_precipitation_mm,origin_humidity,origin_visibility,origin_cloud_cover,origin_pressure,dest_min_temp,dest_total_snow_cm,dest_windspeed_kmph,dest_precipitation_mm,dest_humidity,dest_visibility,dest_cloud_cover,dest_pressure
0,WN,95,2020,1,1,3,18,19,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,1.478261,0.00,42.521739,0.0,9.956522,5.258824,18.421053,0.0,25.368421,0.0,26.473684,7086.0,57334.0,152.074203,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,11.0,0.0,12.0,0.0,83.0,10.0,29.0,1021.0
1,WN,90,2020,1,1,3,11,13,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,1.478261,0.00,42.521739,0.0,9.956522,21.142857,0.000000,0.0,17.500000,0.0,77.000000,7086.0,57334.0,152.074203,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,11.0,0.0,12.0,0.0,83.0,10.0,29.0,1021.0
2,WN,70,2020,1,1,3,20,21,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,8.811881,7.871429,0.0,2.728571,0.0,47.800000,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0
3,WN,75,2020,1,1,3,13,14,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,2.074074,15.717391,0.0,2.239130,0.0,21.956522,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0
4,WN,80,2020,1,1,3,9,10,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,-2.740260,75.000000,0.0,17.833333,0.0,13.000000,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0


## Feature scale and encode

In [14]:
from src.modules.feature_engineering import encoder

In [15]:
scaled_df = encoder(X_df, drop_first=False)
print(scaled_df.shape)
scaled_df.head()

(150623, 56)


,crs_elapsed_time,year,month,day,day_of_week,crs_dep_hour,crs_arr_hour,month_avg_arr_delay,month_avg_carrier_delay,month_avg_weather_delay,month_avg_nas_delay,month_avg_security_delay,month_avg_late_aircraft_delay,day_of_week_avg_arr_delay,day_of_week_avg_carrier_delay,day_of_week_avg_weather_delay,day_of_week_avg_nas_delay,day_of_week_avg_security_delay,day_of_week_avg_late_aircraft_delay,arr_hour_avg_arr_delay,arr_hour_avg_carrier_delay,arr_hour_avg_weather_delay,arr_hour_avg_nas_delay,arr_hour_avg_security_delay,arr_hour_avg_late_aircraft_delay,origin_total_flights,dest_total_flights,month_flight_avg_seats,carrier_month_seats,carrier_month_avg_seats,origin_min_temp,origin_total_snow_cm,origin_windspeed_kmph,origin_precipitation_mm,origin_humidity,origin_visibility,origin_cloud_cover,origin_pressure,dest_min_temp,dest_total_snow_cm,dest_windspeed_kmph,dest_precipitation_mm,dest_humidity,dest_visibility,dest_cloud_cover,dest_pressure,mkt_unique_carrier_AA,mkt_unique_carrier_AS,mkt_unique_carrier_B6,mkt_unique_carrier_DL,mkt_unique_carrier_F9,mkt_unique_carrier_G4,mkt_unique_carrier_HA,mkt_unique_carrier_NK,mkt_unique_carrier_UA,mkt_unique_carrier_WN
0,95,2020,1,1,3,18,19,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,1.478261,0.00,42.521739,0.0,9.956522,5.258824,18.421053,0.0,25.368421,0.0,26.473684,7086.0,57334.0,152.074203,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,11.0,0.0,12.0,0.0,83.0,10.0,29.0,1021.0,0,0,0,0,0,0,0,0,0,1
1,90,2020,1,1,3,11,13,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,3.125000,1.478261,0.00,42.521739,0.0,9.956522,21.142857,0.000000,0.0,17.500000,0.0,77.000000,7086.0,57334.0,152.074203,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,11.0,0.0,12.0,0.0,83.0,10.0,29.0,1021.0,0,0,0,0,0,0,0,0,0,1
2,70,2020,1,1,3,20,21,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,8.811881,7.871429,0.0,2.728571,0.0,47.800000,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0,0,0,0,0,0,0,0,0,0,1
3,75,2020,1,1,3,13,14,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,2.074074,15.717391,0.0,2.239130,0.0,21.956522,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0,0,0,0,0,0,0,0,0,0,1
4,80,2020,1,1,3,9,10,-0.467742,9.846154,0.0,7.128205,1.948718,20.102564,-1.183761,9.200000,0.72,3.920000,0.0,33.140000,-2.740260,75.000000,0.0,17.833333,0.0,13.000000,7086.0,20962.0,155.201183,79269676,154.052359,10.0,0.0,4.0,0.0,34.0,10.0,5.0,1017.0,8.0,0.0,6.0,0.0,71.0,10.0,44.0,1021.0,0,0,0,0,0,0,0,0,0,1


In [16]:
from src.modules.models import regmodel_evaluation

In [17]:
X = scaled_df.values
X.shape

(150623, 56)

In [18]:
arr_delay_predictions = xgb_model['XGBoost_Reg'].predict(X)
print(arr_delay_predictions[:50])

[323.52313 312.78128 340.47223 337.21356 330.76984 302.32352 340.21472
 327.15256 317.80765 312.83606 320.70883 330.41116 325.78235 328.93198
 355.02084 349.0824  364.72318 361.48593 357.27847 351.95605 303.77042
 271.22388 311.30692 299.08582 319.19064 351.40097 323.3974  324.045
 340.08075 349.42566 349.44705 320.34793 307.20114 312.37497 304.5241
 296.65134 355.00244 318.07013 323.92868 352.9694  314.48828 324.2887
 320.0651  314.9224  280.08676 349.93085 311.3316  330.1739  331.64542
 327.6034 ]


In [10]:
arr_delay_predictions.mean()

337.95517

In [11]:
arr_delay_predictions.max()

431.61887

In [12]:
arr_delay_predictions.min()

176.35841

In [13]:
np.median(arr_delay_predictions)

340.99054

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_pred, y_test)
ax.plot([y_pred.min(), y_pred.max()], [y_pred.min(), y_pred.max()], ‘k--’, lw=4)
ax.set_xlabel(‘Predicted’)
ax.set_ylabel(‘Measured’)
plt.show()